## Log
- Written by Junhyeon Kang / email:junhyeon@tesser.co.kr
- Written date: 20230722

---------

## Code Description
- Visualize predictions on LNDb CT images

-------

## Code Flow
- Load CT Data as 3D Arrays
- Draw tumor boxes based on predictions
- Save CT Data as NIFTI files

---------

## Package Import

In [ ]:
# data science
import os, glob, shutil
import numpy as np
from numpy.linalg import inv
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pickle

# Medical Image package
import nibabel as nib
from nibabel.affines import apply_affine
import pydicom
import skimage.io as io

# etc
from tqdm import tqdm
import ipywidgets as widgets
from datetime import date

-----------

## Load PKL predictions

In [ ]:
pkl_path = '/home/ubuntu/junhyeon/junhyeon/Task101_LNDbunion/RetinaUNetV001_D3V001_3d/fold0/val_predictions/'
pkl_list = os.listdir(pkl_path)
mask_filedic = {}

for pkl in pkl_list:
    with open(pkl_path+pkl, 'rb') as f:
        data = pickle.load(f)
        box_coords = data['pred_boxes']
        box_scores = data['pred_scores']
        score_thresh = 0.4
        result_indices = np.where(box_scores>score_thresh)
        results = box_coords[result_indices]
        mask_filedic[pkl.split('_')[0]] = results
        
mask_filedic

------------

## Util Functions

In [ ]:
def load_nii(file_path):
    """
    load nii file
    
    Args:
        file_path (str): nii file path
    Returns : 
        nii (nibabel.nifti1.Nifti1Image): NIFTI file
        affine (numpy.ndarray): affine matrix
        header (dict): header information
    
    """
    nii = nib.load(file_path)
    header = nii.header
    affine = nii.affine
    nii = nii.get_fdata()
    
    return nii, affine, header

In [ ]:
def draw_seg(arr, x_tup, y_tup,z_tup):
    """
    
    Args : 
        arr (array): CT image array
        x_tup (tuple): tuple of min and max x coordinates
        y_tup (tuple): tuple of min and max y coordinates
        z_tup (tuple): tuple of min and max z coordinates
        
    Returns : 
        draw_array(array) : Draw CT image with segmentation array
    
    """
    
    segmentation_array = np.matrix.copy(arr)
    
    # x1, x2 = int(x_coor - radius), int(x_coor + radius)
    # y1, y2 = int(y_coor - radius), int(y_coor + radius)
    # z1, z2 = int(z_coor - radius), int(z_coor + radius) # 더 멀리 띄워서 잘 보이게? 지금 상황이면 가장 윗면은 하얀 사각형으로 보일것.
    

    x1, x2 = int(x_tup[0]), int(x_tup[1])
    y1, y2 = int(y_tup[0]), int(y_tup[1])
    z1, z2 = int(z_tup[0]), int(z_tup[1])

    whitest = arr.max()
        
    segmentation_array[x1:x2+1, y1, z1:z2+1] = whitest
    segmentation_array[x1:x2+1, y2, z1:z2+1] = whitest
    segmentation_array[x1, y1:y2+1, z1:z2+1] = whitest
    segmentation_array[x2, y1:y2+1, z1:z2+1] = whitest

    return segmentation_array

---------

## Nii File paths (on which to draw)

In [ ]:
nii_file_paths = os.listdir('/mnt/tesser_nas2/AI_DataSets/lg_tm_CT_LNDb/nii/data_nii/')
nii_file_paths

-----------

## Print masks

In [ ]:
def printer(filename):
    """
    Args : 
        filename (str): LNDb file name - ex) LNDb-0013
        
    Returns : 
        Saves bbox images (.nii.gz) in nii_nndetpredictions folder
    """
    nii, affine, header = load_nii('/mnt/tesser_nas2/AI_DataSets/lg_tm_CT_LNDb/nii/data_nii/' + filename + '.nii.gz')

    masklist = mask_filedic[filename]
    
    seg = nii # initial value, will change each iteration below
    for mask in masklist:
        # mask coordinates: z1, y1, z2, y2, x1, x2
        z_tup = (mask[0], mask[2])
        y_tup = (mask[1], mask[3])
        x_tup = (mask[4], mask[5])
        
        try:
            seg = draw_seg(seg, x_tup, y_tup, z_tup)
        except:
            print(filename, 'There exists a prediction out of bounds')

    ni_img = nib.Nifti1Image(seg, affine=affine)
    nib.save(ni_img, os.path.join("/mnt/tesser_nas2/AI_DataSets/lg_tm_CT_LNDb/nii_nndetpredictions/", filename + "_bbox.nii.gz"))

In [ ]:
# Run on all predictions

for filename in os.listdir('/home/ubuntu/junhyeon/junhyeon/Task101_LNDbunion/RetinaUNetV001_D3V001_3d/fold0/val_predictions'):
    printer(filename.split('_')[0])